## Encontrando um modelo de classificacao pra definir a tipificação do delito.

## 1- Importando a base de dados

In [1]:
#  Importando as bibliotecas

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date

 - Link do site onde encontra-se o dataset
 
 url:https://data.cityofnewyork.us/Public-Safety/NYPD-Arrest-Data-Year-to-Date-/uip8-fykc

In [2]:
df=pd.read_csv('dataset.csv')

In [3]:
# visualizando as 5 primeiras linhas do dataset
df[0:5]

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,218375425,09/26/2020,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,M,1,0,25-44,M,WHITE,982285,201682,40.720255,-74.007090,POINT (-74.00709027999993 40.72025522300004)
1,217319714,09/02/2020,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,M,5,0,18-24,F,BLACK,983903,200257,40.716344,-74.001253,POINT (-74.00125319299997 40.71634415200003)
2,218282529,09/24/2020,244.0,"BURGLARY,UNCLASSIFIED,UNKNOWN",107.0,BURGLARY,PL 1402000,F,M,34,0,45-64,M,BLACK,1003633,253391,40.862161,-73.929927,POINT (-73.92992712599995 40.862161439000026)
3,217526617,09/07/2020,792.0,WEAPONS POSSESSION 1 & 2,118.0,DANGEROUS WEAPONS,PL 2650303,F,K,67,0,18-24,M,BLACK,1003029,176696,40.651655,-73.932325,POINT (-73.93232493699998 40.65165467100008)
4,218518385,09/29/2020,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,K,66,0,18-24,M,BLACK,988164,170927,40.635839,-73.985899,POINT (-73.98589935799998 40.635839049000026)


## 2- Preparaçao dos dados


 - Removendo as colunas que nao serão utilizadas 

In [4]:
dados=df.drop(['ARREST_KEY', 'PD_CD', 'PD_DESC', 'KY_CD', 'LAW_CODE','ARREST_PRECINCT', 'JURISDICTION_CODE', 'X_COORD_CD', 'Y_COORD_CD','New Georeferenced Column'], axis=1)

In [5]:
#  Renomeando as colunas

dados.columns = ['data_da_ocorrencia','descricao_da_ofensa','nivel_da_ofensa','bairro'
                     ,'idade_do_infrator','sexo_do_infrator','raca_do_infrator','latitude','longitude']

In [6]:
# reorganizando os dados referentes a data 

def date_to_weekday(date):
    weekday_dict = {0:'Monday', 1:'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    date_time_obj = datetime.datetime.strptime(date, '%m/%d/%Y')
    return weekday_dict[date_time_obj.weekday()]

date = dados['data_da_ocorrencia'].str.split("/", n = 3, expand = True)
dados['ano'] = date[2].astype('int32')
dados['dia'] = date[1].astype('int32')
dados['mes'] = date[0].astype('int32')
dados['dia_da_semana'] = dados['data_da_ocorrencia'].apply(date_to_weekday)

# alterando os dados da coluna nivel_da_ofensa

new_nivel_da_ofensa = {'M': "contravencao", 'F': "crime", 'V': "violacao",'I':"outro" }
dados['nivel_da_ofensa'] = dados['nivel_da_ofensa'].map(new_nivel_da_ofensa)

# alterando os dados da coluna bairro

new_bairro = {'B': 'Bronx', 'S': 'Staten_Island', 'K': 'Brooklyn', 'M': 'Manhattan' , 'Q': 'Queens' }
dados['bairro'] = dados['bairro'].map(new_bairro)

In [7]:
dados.head()

,data_da_ocorrencia,descricao_da_ofensa,nivel_da_ofensa,bairro,idade_do_infrator,sexo_do_infrator,raca_do_infrator,latitude,longitude,ano,dia,mes,dia_da_semana
0,09/26/2020,RAPE,crime,Manhattan,25-44,M,WHITE,40.720255,-74.007090,2020,26,9,Saturday
1,09/02/2020,FORGERY,crime,Manhattan,18-24,F,BLACK,40.716344,-74.001253,2020,2,9,Wednesday
2,09/24/2020,BURGLARY,crime,Manhattan,45-64,M,BLACK,40.862161,-73.929927,2020,24,9,Thursday
3,09/07/2020,DANGEROUS WEAPONS,crime,Brooklyn,18-24,M,BLACK,40.651655,-73.932325,2020,7,9,Monday
4,09/29/2020,CRIMINAL MISCHIEF & RELATED OF,crime,Brooklyn,18-24,M,BLACK,40.635839,-73.985899,2020,29,9,Tuesday


 - Convertendo a coluna " DATA_DA_OCORRENCIA "para formato datetime

In [8]:
dados["data_da_ocorrencia"] = pd.to_datetime(dados["data_da_ocorrencia"] )

## 3.1 Preparação dos Dados para modelagem
- Separação Treino/Teste
- Separar Dados de Entrada (Atributos) e de saída (target)
- Tratar colunas categóricas
- Tratar dados ausentes


## Criando um modelo de classificacao pra definir a faixa etária de quem praticou o delito.

 - Selecionando apenas os delitos mais recorrentes

In [9]:
# dados = dados.query('descricao_da_ofensa =="ASSAULT 3 & RELATED OFFENSES" or  descricao_da_ofensa =="FELONY ASSAULT" or descricao_da_ofensa =="PETIT LARCENY" or descricao_da_ofensa =="DANGEROUS DRUGS" or descricao_da_ofensa =="MISCELLANEOUS PENAL LAW" or descricao_da_ofensa =="ROBBERY" or descricao_da_ofensa =="CRIMINAL MISCHIEF & RELATED OF" or descricao_da_ofensa =="BURGLARY" or  descricao_da_ofensa =="GRAND LARCENY" or descricao_da_ofensa =="DANGEROUS WEAPONS" or  descricao_da_ofensa =="OFFENSES AGAINST PUBLIC ADMINI" or descricao_da_ofensa =="VEHICLE AND TRAFFIC LAWS" or  descricao_da_ofensa =="OFF. AGNST PUB ORD SENSBLTY &" or descricao_da_ofensa =="SEX CRIMES" or descricao_da_ofensa =="INTOXICATED & IMPAIRED DRIVING"  ')

 - Verificando a exitência de dados ausentes>

In [10]:
dados.isna().sum()

data_da_ocorrencia       0
descricao_da_ofensa     24
nivel_da_ofensa        970
bairro                   0
idade_do_infrator        0
sexo_do_infrator         0
raca_do_infrator         0
latitude                 0
longitude                0
ano                      0
dia                      0
mes                      0
dia_da_semana            0
dtype: int64

In [11]:
dados = dados.dropna()

In [12]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102382 entries, 0 to 103375
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   data_da_ocorrencia   102382 non-null  datetime64[ns]
 1   descricao_da_ofensa  102382 non-null  object        
 2   nivel_da_ofensa      102382 non-null  object        
 3   bairro               102382 non-null  object        
 4   idade_do_infrator    102382 non-null  object        
 5   sexo_do_infrator     102382 non-null  object        
 6   raca_do_infrator     102382 non-null  object        
 7   latitude             102382 non-null  float64       
 8   longitude            102382 non-null  float64       
 9   ano                  102382 non-null  int32         
 10  dia                  102382 non-null  int32         
 11  mes                  102382 non-null  int32         
 12  dia_da_semana        102382 non-null  object        
dtypes: datetime64[

 - Excluindo colunas que não serão utilizadas.
 
 Como todos os delitos são referentes ao ano de 2020 , iremos excluir apenas essa coluna.

In [13]:
dados = dados.drop(columns=['ano'])

 - Convertendo variáveis categóricas em variáveis numéricas

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
enconder = LabelEncoder()

In [16]:
dados['data_da_ocorrencia'] = enconder.fit_transform(dados['data_da_ocorrencia'])
dados['nivel_da_ofensa'] = enconder.fit_transform(dados['nivel_da_ofensa'])
dados['idade_do_infrator'] = enconder.fit_transform(dados['idade_do_infrator'])
dados['descricao_da_ofensa'] = enconder.fit_transform(dados['descricao_da_ofensa'])
dados['bairro'] = enconder.fit_transform(dados['bairro'])
dados['sexo_do_infrator'] = enconder.fit_transform(dados['sexo_do_infrator'])
dados['raca_do_infrator'] = enconder.fit_transform(dados['raca_do_infrator'])
dados['mes'] = enconder.fit_transform(dados['mes'])
dados['dia'] = enconder.fit_transform(dados['dia'])
dados['dia_da_semana'] = enconder.fit_transform(dados['dia_da_semana'])
dados['latitude'] = enconder.fit_transform(dados['latitude'])
dados['longitude'] = enconder.fit_transform(dados['longitude'])

In [17]:
dados.head()

,data_da_ocorrencia,descricao_da_ofensa,nivel_da_ofensa,bairro,idade_do_infrator,sexo_do_infrator,raca_do_infrator,latitude,longitude,dia,mes,dia_da_semana
0,269,51,1,2,1,1,5,13731,2311,25,8,2
1,245,17,1,2,0,0,2,13353,2877,1,8,6
2,267,7,1,2,2,1,2,24491,11496,23,8,4
3,250,12,1,1,0,1,2,5409,11220,6,8,1
4,272,9,1,1,0,1,2,3926,4688,28,8,5


Separando as variaveis em X e Y

In [19]:
X = dados.drop('sexo_do_infrator',axis=1).values
Y = dados['sexo_do_infrator'].values


Separando em amostras de treino e teste e reescalando os dados

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_treino,X_teste,Y_treino,Y_teste = train_test_split(X,Y,test_size=0.25,random_state=0)

In [23]:
# pegando DummyClassifier como baseline 

from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(X_treino, Y_treino)
acuracia = dummy_stratified.score(X_teste, Y_teste) * 100

print('A acurácia do dummy stratified foi %.2f%%' % acuracia)

A acurácia do dummy stratified foi 71.97%


In [25]:
# pegando DummyClassifier como baseline 

from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy="most_frequent")
dummy_stratified.fit(X_treino, Y_treino)
acuracia = dummy_stratified.score(X_teste, Y_teste) * 100

print('A acurácia do dummy mostfrequent foi %.2f%%' % acuracia)

A acurácia do dummy mostfrequent foi 83.18%


 - Reescalando os dados e aplicando um modelo

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


SEED = 5
np.random.seed(SEED)
raw_x_treino, raw_x_teste, y_treino, y_teste = train_test_split(X, Y, test_size = 0.25,
                                                         stratify = Y)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(X_treino), len(X_teste)))

scaler = StandardScaler()
scaler.fit(raw_x_treino)
x_treino = scaler.transform(raw_x_treino)
x_teste = scaler.transform(raw_x_teste)

modelo = SVC()
modelo.fit(x_treino, y_treino)
previsoes = modelo.predict(x_teste)
acuracia = accuracy_score(y_teste, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 76786 elementos e testaremos com 25596 elementos
